# Preprocessing Workshop

![Your Brain on Sci-Kit Learn by David Llanio](https://i.imgur.com/FpwfKKo.png)

## Import and Manipulate our data

In [ ]:
import pandas as pd
import numpy as np
import sklearn

df1 = pd.read_csv('/work/2019 - 01.csv')
df2 = pd.read_csv('/work/2019 - 02.csv')
df3 = pd.read_csv('/work/2019 - 03.csv')
df4 = pd.read_csv('/work/2019 - 04.csv')
df5 = pd.read_csv('/work/2019 - 05.csv')

In [ ]:
#creating the mega dataframe

frames = [df1, df2, df3, df4, df5]
dfmega = pd.concat(frames)


## Handling missing values

In [ ]:
dfmega_missing = dfmega.isnull().sum()

# Make sure we are looking at all the values in the df
dfmega_missing[:]

month                     0
trip_duration_sec         0
start_station_id        745
start_station_name      745
end_station_id          745
end_station_name        745
bike_id                   0
user_type                 0
member_birth_year     49376
member_gender         49370
dtype: int64

In [ ]:
# Drop the columns we don't need/are unreliable
dfmega_updated = dfmega.drop(
                columns=['start_station_id', 'end_station_id','bike_id',
                'member_birth_year','end_station_name','member_gender'])

dfmega_updated.head(10)

,month,trip_duration_sec,start_station_name,user_type
0,January,80825,Foothill Blvd at 42nd Ave,Subscriber
1,January,65900,Cyril Magnin St at Ellis St,Subscriber
2,January,62633,Downtown Berkeley BART,Customer
3,January,44680,Church St at Duboce Ave,Customer
4,January,60709,Steuart St at Market St,Customer
5,January,6733,Downtown Berkeley BART,Subscriber
6,January,1188,Father Alfred E Boeddeker Park,Subscriber
7,January,1254,San Carlos St at Market St,Subscriber
8,January,3153,O'Farrell St at Divisadero St,Subscriber
9,January,323,16th St Mission BART Station 2,Subscriber


In [ ]:
dfmega_missing = dfmega_updated.isnull().sum() #updated df w/o id columns, etc.

# Make sure we are looking at all the values in the df
dfmega_missing[:]

month                   0
trip_duration_sec       0
start_station_name    745
user_type               0
dtype: int64

### Removing Missing values

In [ ]:
# If you want to drop all rows with missing values
df_dropna = dfmega_missing.dropna()
print(df_dropna.isnull().sum())


0


### Imputing missing values
Imputing is the filling of values

In [ ]:
# Find the missing values to determine which to use
na_values = dfmega_updated.isnull().any(axis=1)
dfmega_updated[na_values]

# Slice of our data for tutorial purposes
dfmega_updated.iloc[2020:2026]

,month,trip_duration_sec,start_station_name,user_type
2020,January,905,San Francisco Caltrain (Townsend St at 4th St),Subscriber
2021,January,193,Folsom St at 9th St,Subscriber
2022,January,109,Telegraph Ave at 19th St,Subscriber
2023,January,746,NaN,Subscriber
2024,January,196,Folsom St at 9th St,Subscriber
2025,January,400,Townsend St at 5th St,Customer


In [ ]:
# Let's impute each missing value as a 0 using Panda's fillna() function
df_zero_na = dfmega_updated.fillna(0)

print(df_zero_na.isnull().sum())

month                 0
trip_duration_sec     0
start_station_name    0
user_type             0
dtype: int64


In [ ]:
# Let's check what happened to row 2023
df_zero_na.iloc[2020:2026]

,month,trip_duration_sec,start_station_name,user_type
2020,January,905,San Francisco Caltrain (Townsend St at 4th St),Subscriber
2021,January,193,Folsom St at 9th St,Subscriber
2022,January,109,Telegraph Ave at 19th St,Subscriber
2023,January,746,0,Subscriber
2024,January,196,Folsom St at 9th St,Subscriber
2025,January,400,Townsend St at 5th St,Customer


#### Types of Methods in fillna()
According to the [pandas docs](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html)

backfill/bfill: Takes value from next valid observation

pad/ffill: Takes value from prior valid observation


In [ ]:
# Replacing missing values with the proceeding value 
df_replacena = dfmega_updated.fillna(method='bfill', axis=0)

In [ ]:
# Let's check what happened to row 2023
df_replacena.iloc[2020:2026]

,month,trip_duration_sec,start_station_name,user_type
2020,January,905,San Francisco Caltrain (Townsend St at 4th St),Subscriber
2021,January,193,Folsom St at 9th St,Subscriber
2022,January,109,Telegraph Ave at 19th St,Subscriber
2023,January,746,Folsom St at 9th St,Subscriber
2024,January,196,Folsom St at 9th St,Subscriber
2025,January,400,Townsend St at 5th St,Customer


#### Note:
Imputing values using the proceeding value is normally useful when data has a logical order to it.


## Splitting the labels

In [ ]:
#Seperate the labels to be predicted from the rest of the data

X = df_replacena.copy()
y = X['trip_duration_sec']

X.drop(columns=['trip_duration_sec'],inplace=True)

In [ ]:
X.head()

,month,start_station_name,user_type
0,January,Foothill Blvd at 42nd Ave,Subscriber
1,January,Cyril Magnin St at Ellis St,Subscriber
2,January,Downtown Berkeley BART,Customer
3,January,Church St at Duboce Ave,Customer
4,January,Steuart St at Market St,Customer


In [ ]:
y.head()

0    80825
1    65900
2    62633
3    44680
4    60709
Name: trip_duration_sec, dtype: int64

## Handling Numerical Data

Reference: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html 

In [ ]:
# Light intensity data
lightdata = {'name':['flashlight','sun','glow_stick','lamp','neon_sign','led'],'lumens_W':[24,999,15,38,52,5]}
lightdf = pd.DataFrame(data=lightdata)


# Kitchen and living room size
kitlivdata = {'kitchen_size':[425,1500,54,89,250,7000,999],'living_room_size':[722,422,1500,8000,52,2000,999]}
kitlivdf = pd.DataFrame(data=kitlivdata)

In [ ]:
lightdf

,name,lumens_W
0,flashlight,24
1,sun,999
2,glow_stick,15
3,lamp,38
4,neon_sign,52
5,led,5


In [ ]:
kitlivdf

,kitchen_size,living_room_size
0,425,722
1,1500,422
2,54,1500
3,89,8000
4,250,52
5,7000,2000
6,999,999


### Normalizing the data the simple way

In [ ]:
lightdf['lumens_W'] = lightdf['lumens_W'].apply(lambda x: x/999)

In [ ]:
lightdf

,name,lumens_W
0,flashlight,0.024024
1,sun,1.000000
2,glow_stick,0.015015
3,lamp,0.038038
4,neon_sign,0.052052
5,led,0.005005


### Normalizing the data the Sci-kit learn way

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [ ]:
print(scaler.fit(kitlivdf))
print("")
print(scaler.mean_)
print("")
print(scaler.transform(kitlivdf))

StandardScaler()

[1473.85714286 1956.42857143]

[[-0.45439463 -0.48601926]
 [ 0.01132583 -0.60413528]
 [-0.61512234 -0.17970507]
 [-0.59995935  2.37947518]
 [-0.53020959 -0.74981169]
 [ 2.39408165  0.01715494]
 [-0.20572157 -0.37695881]]


## Handling Categorical Data

In [ ]:
#Let's take a look at the dataframe

X.head()


,month,start_station_name,user_type
0,January,Foothill Blvd at 42nd Ave,Subscriber
1,January,Cyril Magnin St at Ellis St,Subscriber
2,January,Downtown Berkeley BART,Customer
3,January,Church St at Duboce Ave,Customer
4,January,Steuart St at Market St,Customer


In [ ]:
pd.unique(X['month'])

array(['January', 'Februrary', 'March', 'April', 'May'], dtype=object)

### Switch the categorical values with numbers

In [ ]:
#Let's switch the months for their correspinding number
#January = 1, February = 2, March = 3, April = 4, May = 5

cleanup_months = {"month": {"January": 1, "Februrary": 2, "March": 3, "April": 4, "May": 5}}
X = X.replace(cleanup_months)
X.head()

,month,start_station_name,user_type
0,1,Foothill Blvd at 42nd Ave,Subscriber
1,1,Cyril Magnin St at Ellis St,Subscriber
2,1,Downtown Berkeley BART,Customer
3,1,Church St at Duboce Ave,Customer
4,1,Steuart St at Market St,Customer


In [ ]:
pd.unique(X['month'])

array([1, 2, 3, 4, 5])

### One hot encoding

Reference: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

![onehot diagram](https://i.imgur.com/mtimFxh.png)

In [ ]:
X = df_replacena.copy()
y = X['trip_duration_sec']

X.drop(columns=['trip_duration_sec'],inplace=True)

In [ ]:
X.head()

,month,start_station_name,user_type
0,January,Foothill Blvd at 42nd Ave,Subscriber
1,January,Cyril Magnin St at Ellis St,Subscriber
2,January,Downtown Berkeley BART,Customer
3,January,Church St at Duboce Ave,Customer
4,January,Steuart St at Market St,Customer


In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown='error',drop='if_binary')
encoder.fit(X)

OneHotEncoder(drop='if_binary')

In [ ]:
print(encoder.transform(X).toarray())

[[0. 0. 1. ... 0. 0. 1.]
 [0. 0. 1. ... 0. 0. 1.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


In [ ]:
encoder.get_feature_names()

array(['x0_April', 'x0_Februrary', 'x0_January', 'x0_March', 'x0_May',
       'x1_10th Ave at E 15th St', 'x1_10th St at Empire St',
       'x1_10th St at Fallon St', 'x1_10th St at Mission St',
       'x1_10th St at University Ave', 'x1_11th St at Bryant St',
       'x1_11th St at Natoma St', 'x1_13th St at Franklin St',
       'x1_14th St at Filbert St', 'x1_14th St at Mandela Pkwy',
       'x1_14th St at Mission St', 'x1_15th St at Potrero Ave',
       'x1_16th St Depot', 'x1_16th St Mission BART',
       'x1_16th St Mission BART Station 2', 'x1_16th St at Prosper St',
       'x1_17th & Folsom Street Park (17th St at Folsom St)',
       'x1_17th St at Dolores St', 'x1_17th St at Santa Clara St',
       'x1_17th St at Valencia St', 'x1_18th St at Noe St',
       'x1_19th St at Florida St', 'x1_19th St at Mission St',
       'x1_19th Street BART Station', 'x1_1st St at Folsom St',
       'x1_1st St at San Carlos Ave', 'x1_1st St at Younger Ave',
       'x1_20th St at Bryant St', 'x1_2

In [ ]:
X_pd= pd.get_dummies(X)

In [ ]:
X_pd.head()

,month_April,month_Februrary,month_January,month_March,month_May,start_station_name_10th Ave at E 15th St,start_station_name_10th St at Empire St,start_station_name_10th St at Fallon St,start_station_name_10th St at Mission St,start_station_name_10th St at University Ave,...,start_station_name_West St at University Ave,start_station_name_William St at 10th St,start_station_name_Williams Ave at 3rd St,start_station_name_Williams Ave at Apollo St,start_station_name_Willow St at Vine St,start_station_name_Woolsey St at Sacramento St,start_station_name_Woolsey St at Sacramento St1,start_station_name_Yerba Buena Center for the Arts (Howard St at 3rd St),user_type_Customer,user_type_Subscriber
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


## Splitting the data (train/test)

Reference: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [ ]:
X = df_replacena.copy()
y = X['trip_duration_sec']

X.drop(columns=['trip_duration_sec'],inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"X shape: {X.shape}")

X_train shape: (842453, 3)
X_test shape: (210614, 3)
X shape: (1053067, 3)


## Data Pipeline

Reference: https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html 

In [ ]:
#Importing tools for pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

#Define the numerical columns
numerical_cols = []

#Define the categorical columns
categorical_cols = ['month', 'start_station_name','user_type']


#Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

#Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

#Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)

In [ ]:
from sklearn.metrics import mean_absolute_error
import joblib

# # Bundle preprocessing and modeling code in a pipeline
# my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
#                               ('model', model)
#                              ])

# # Preprocessing of training data, fit model 
# my_pipeline.fit(X_train, y_train)


my_pipeline = joblib.load('/work/my_pipeline.pickle')

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_test)

# Evaluate the model
score = mean_absolute_error(y_test, preds)
print('MAE:', score)

MAE: 466.82790595395005


In [ ]:
# {"month": {"January": 1, "Februrary": 2, "March": 3, "April": 4, "May": 5}}

data_mu = {'month':['January','May','March','March','May'],'start_station_name':['Myrtle St at Polk St','San Carlos St at Market St','Auzerais Ave at Lincoln Ave','Auzerais Ave at Lincoln Ave','Auzerais Ave at Lincoln Ave'],'user_type':['Customer','Subscriber','Customer','Subscriber','Customer']}

In [ ]:
data_mu

{'month': ['January', 'May', 'March', 'March', 'May'],
 'start_station_name': ['Myrtle St at Polk St',
  'San Carlos St at Market St',
  'Auzerais Ave at Lincoln Ave',
  'Auzerais Ave at Lincoln Ave',
  'Auzerais Ave at Lincoln Ave'],
 'user_type': ['Customer', 'Subscriber', 'Customer', 'Subscriber', 'Customer']}

In [ ]:
mudf = pd.DataFrame(data=data_mu)

In [ ]:
mudf

,month,start_station_name,user_type
0,January,Myrtle St at Polk St,Customer
1,May,San Carlos St at Market St,Subscriber
2,March,Auzerais Ave at Lincoln Ave,Customer
3,March,Auzerais Ave at Lincoln Ave,Subscriber
4,May,Auzerais Ave at Lincoln Ave,Customer


In [ ]:
preds = my_pipeline.predict(mudf)

In [ ]:
print(preds)

[1547.49198919  673.08780928 1026.20122326  604.97679868  862.50482079]


In [ ]:
# joblib.dump(my_pipeline, 'my_pipeline.pickle')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=092fbd09-8de9-42d2-853c-f1bced3014bf' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>